<a href="https://colab.research.google.com/github/Akansha9717/TextMining/blob/main/textmining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BIOMEDICAL LITERATURE MINING USING PYTHON
at EduBooster Research & Technology

In [1]:
import argparse

In [2]:
import time

In [3]:
from bs4 import BeautifulSoup

In [4]:
import pandas as pd
import requests


In [5]:
import random

In [6]:
import warnings

In [7]:
import socket

In [8]:
import asyncio

In [9]:
import html5lib
from argparse import ArgumentParser

In [11]:
warnings.filterwarnings('ignore', '.*do not.*',)
user_agents = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) Gecko/20100101 Firefox/55.0",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.101 Safari/537.36",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
        ]


In [12]:
def make_header():
    headers = {
            'User-Agent':random.choice(user_agents),
            }
    return headers


In [13]:
async def extract_by_article(url):
    conn = aiohttp.TCPConnector(family=socket.AF_INET,ssl=False)
    headers = make_header()
    global articles_data
    async with aiohttp.ClientSession(headers=headers, connector=conn) as session:
        async with semaphore, session.get(url) as response:
            data = await response.text()
            soup = BeautifulSoup(data, "lxml")
            try:
                abstract_raw = soup.find('div', {'class': 'abstract-content selected'}).find_all('p')
                abstract = ' '.join([paragraph.text.strip() for paragraph in abstract_raw])
            except:
                abstract = 'NO_ABSTRACT'
            try:
                all_affiliations = soup.find('ul', {'class':'item-list'}).find_all('li')
                for affiliation in all_affiliations:
                    affiliations.append(affiliation.get_text().strip())
            except:
                affiliations = 'NO_AFFILIATIONS'
            try:
                has_keywords = soup.find_all('strong',{'class':'sub-title'})[-1].text.strip()
                if has_keywords == 'Keywords:':
                    keywords = soup.find('div', {'class':'abstract' }).find_all('p')[-1].get_text()
                    keywords = keywords.replace('Keywords:','\n').strip() 
                else:
                    keywords = 'NO_KEYWORDS'
            except:
                keywords = 'NO_KEYWORDS'
            try:
                title = soup.find('meta',{'name':'citation_title'})['content'].strip('[]')
            except:
                title = 'NO_TITLE'
            authors = ''   
            try:
                for author in soup.find('div',{'class':'authors-list'}).find_all('a',{'class':'full-name'}):
                    authors += author.text + ', '
            except:
                authors = ('NO_AUTHOR')
            try:
                journal = soup.find('meta',{'name':'citation_journal_title'})['content']
            except:
                journal = 'NO_JOURNAL'
            try:
                date = soup.find('time', {'class': 'citation-year'}).text
            except:
                date = 'NO_DATE'

            article_data = {
                'url': url,
                'title': title,
                'authors': authors,
                'abstract': abstract,
                'affiliations': affiliations,
                'journal': journal,
                'keywords': keywords,
                'date': date
            }
            articles_data.append(article_data)


In [14]:
async def get_pmids(page, keyword):
    page_url = f'{pubmed_url}+{keyword}+&page={page}'
    headers = make_header()
    async with aiohttp.ClientSession(headers=headers) as session:
        async with session.get(page_url) as response:
            data = await response.text()
            soup = BeautifulSoup(data, "lxml")
            pmids = soup.find('meta',{'name':'log_displayeduids'})['content']
            for pmid in pmids.split(','):
                url = root_pubmed_url + '/' + pmid
                urls.append(url)

In [15]:
def get_num_pages(keyword):
    if args.pages != None: return args.pages
    headers=make_header()
    search_url = f'{pubmed_url}+{keyword}'
    with requests.get(search_url,headers=headers) as response:
        data = response.text
        soup = BeautifulSoup(data, "lxml")
        num_pages = int((soup.find('span', {'class': 'total-pages'}).get_text()).replace(',',''))
        return num_pages 


In [16]:
async def build_article_urls(keywords):
    tasks = []
    for keyword in keywords:
        num_pages = get_num_pages(keyword)
        for page in range(1,num_pages+1):
            task = asyncio.create_task(get_pmids(page, keyword))
            tasks.append(task)

    await asyncio.gather(*tasks)


In [17]:
async def get_article_data(urls):
    tasks = []
    for url in urls:
        if url not in scraped_urls:
            task = asyncio.create_task(extract_by_article(url))
            tasks.append(task)
            scraped_urls.append(url)

    await asyncio.gather(*tasks)

In [18]:
if __name__ == "__main__":
    parser = ArgumentParser(description='Asynchronous PubMed Scraper')
    parser.add_argument('-f')
    parser.add_argument('--pages', type=int, default=None, help='Specify number of pages to scrape for EACH keyword. Each page of PubMed results contains 10 articles. \n Default = all pages returned for all keywords.')
    parser.add_argument('--start', type=int, default=2020, help='Specify start year for publication date range to scrape. Default = 2020')
    parser.add_argument('--stop', type=int, default=2021, help='Specify stop year for publication date range to scrape. Default = 2021')
    parser.add_argument('--output', type=str, default='articles.csv',help='Choose output file name. Default = "articles.csv".')
    args = parser.parse_args()
    if args.output[-4:] != '.csv': args.output += '.csv' 
    from datetime import datetime
    start_time = datetime.now()
    pubmed_url = f'https://pubmed.ncbi.nlm.nih.gov/?term={args.start}%3A{args.stop}%5Bdp%5D'
    root_pubmed_url = 'https://pubmed.ncbi.nlm.nih.gov'
    from google.colab import files
    uploaded = files.upload()  
    search_keywords = []
    with open('keywords.txt') as file:
        keywords = file.readlines()
        [search_keywords.append(keyword.strip()) for keyword in keywords]
    print(f'\nFinding PubMed article URLs for {len(keywords)} keywords found in keywords.txt\n')



Saving keywords.txt to keywords.txt

Finding PubMed article URLs for 1 keywords found in keywords.txt



In [19]:
 articles_data = []

In [20]:
 urls = []
   
 scraped_urls = []

    
 semaphore = asyncio.BoundedSemaphore(100)

In [21]:
import nest_asyncio
nest_asyncio.apply()
import asyncio

In [22]:
loop = asyncio.get_event_loop()

In [25]:
 !pip3 install aiohttp


     |████████████████████████████████| 1.3 MB 5.0 MB/s 
     |████████████████████████████████| 294 kB 33.6 MB/s 
     |████████████████████████████████| 142 kB 52.9 MB/s 


In [26]:
import asyncio
from aiohttp import web
import nest_asyncio
nest_asyncio.apply()

In [27]:
from aiohttp import web
routes = web.RouteTableDef()

In [28]:
import aiohttp

In [29]:
loop.run_until_complete(build_article_urls(search_keywords))
print(f'Scraping initiated for {len(urls)} article URLs found from {args.start} to {args.stop}\n')
loop = asyncio.get_event_loop()
loop.run_until_complete(get_article_data(urls))

Scraping initiated for 598 article URLs found from 2020 to 2021



In [30]:
articles_df = pd.DataFrame(articles_data, columns=['title','abstract','affiliations','authors','journal','date','keywords','url'])
print('Preview of scraped article data:\n')
print(articles_df.head(5))


Preview of scraped article data:

                                               title  ...                                       url
0  Liver X Receptor Alpha Activation Inhibits Aut...  ...  https://pubmed.ncbi.nlm.nih.gov/32557804
1  Anti-inflammatory effects of Aureusidin in LPS...  ...  https://pubmed.ncbi.nlm.nih.gov/31790703
2  Hippo signaling disruption and ovarian follicl...  ...  https://pubmed.ncbi.nlm.nih.gov/32782158
3  Can Vitamin D and L-Cysteine Co-Supplementatio...  ...  https://pubmed.ncbi.nlm.nih.gov/32659175
4  Pterostilbene prevents LPS-induced early pulmo...  ...  https://pubmed.ncbi.nlm.nih.gov/32377661

[5 rows x 8 columns]


In [31]:
filename = args.output
articles_df.to_csv(filename)

In [32]:
print(f'It took {time.time()} seconds to find {len(urls)} articles; {len(scraped_urls)} unique articles were saved to {filename}')

It took 1628665185.5085616 seconds to find 598 articles; 597 unique articles were saved to articles.csv


In [33]:
data = pd.read_csv("articles.csv")

In [34]:
data.head(10)

,Unnamed: 0,title,abstract,affiliations,authors,journal,date,keywords,url
0,0,Liver X Receptor Alpha Activation Inhibits Aut...,Background and aims:\n \n \n ...,NO_AFFILIATIONS,"Yun Seok Kim, Hyeon Joo Nam, Chang Yeob Han, M...","Hepatology (Baltimore, Md.)",NO_DATE,NO_KEYWORDS,https://pubmed.ncbi.nlm.nih.gov/32557804
1,1,Anti-inflammatory effects of Aureusidin in LPS...,"Aureusidin, a naturally-occurring flavonoid, i...",NO_AFFILIATIONS,"Jie Ren, Dan Su, Lixia Li, Heng Cai, Meiju Zha...",Toxicology and applied pharmacology,2020,Aureusidin; HO-1; Inflammation; MAPKs; NF-κB; ...,https://pubmed.ncbi.nlm.nih.gov/31790703
2,2,Hippo signaling disruption and ovarian follicl...,"The Hippo signaling pathway, which is importan...",NO_AFFILIATIONS,"Aaron J W Hsueh, Kazuhiro Kawamura,",Fertility and sterility,NO_DATE,Hippo signaling; diminished ovarian reserve; d...,https://pubmed.ncbi.nlm.nih.gov/32782158
3,3,Can Vitamin D and L-Cysteine Co-Supplementatio...,Early reports indicate an association between ...,NO_AFFILIATIONS,"Sushil K Jain, Rajesh Parsanathan,",Journal of the American College of Nutrition,NO_DATE,African American; COVID-19; L-cysteine; Vitami...,https://pubmed.ncbi.nlm.nih.gov/32659175
4,4,Pterostilbene prevents LPS-induced early pulmo...,Early pulmonary fibrosis after acute lung inju...,NO_AFFILIATIONS,"Huahong Yang, Cong Hua, Xiaolin Yang, Xiaoye F...",Food & function,2020,NO_KEYWORDS,https://pubmed.ncbi.nlm.nih.gov/32377661
5,5,"Molybdenum cofactor biology, evolution and def...",The molybdenum cofactor (Moco) represents an a...,NO_AFFILIATIONS,"Simon J Mayr, Ralf-R Mendel, Guenter Schwarz,",Biochimica et biophysica acta. Molecular cell ...,NO_DATE,Alternative splicing; Cysteine catabolism; Inh...,https://pubmed.ncbi.nlm.nih.gov/33017596
6,6,The Biosynthetic Pathway of Ergothioneine in C...,Ergothioneine is a natural 2-thiol-amidazole a...,NO_AFFILIATIONS,"Xueqin Yang, Shuoxin Lin, Jinde Lin, Yuyao Wan...",International journal of medicinal mushrooms,NO_DATE,NO_KEYWORDS,https://pubmed.ncbi.nlm.nih.gov/32479005
7,7,Maintenance of photosynthetic capacity in floo...,Ethylene is considered one of the most importa...,NO_AFFILIATIONS,"Leandro Federico De Pedro, Francesco Mignolli,...",Physiologia plantarum,NO_DATE,NO_KEYWORDS,https://pubmed.ncbi.nlm.nih.gov/32458443
8,8,Defects in protein folding in congenital hypot...,Primary congenital hypothyroidism (CH) is the ...,NO_AFFILIATIONS,"Héctor M Targovnik, Karen G Scheps, Carina M R...",Molecular and cellular endocrinology,2020,Congenital hypothyroidism; ERAD; Protein foldi...,https://pubmed.ncbi.nlm.nih.gov/31751626
9,9,Lysosomal sulfatases: a growing family,Sulfatases constitute a family of enzymes that...,NO_AFFILIATIONS,"Torben Lübke, Markus Damme,",The Biochemical journal,2020,formyl glycine-generating enzyme; glycosaminog...,https://pubmed.ncbi.nlm.nih.gov/33120425


In [35]:
articles_df.shape

(597, 8)

In [36]:
articles_df['abstract'][1]

'Aureusidin, a naturally-occurring flavonoid, is found in various plants of Cyperaceae such as Heleocharis dulcis (Burm. f.) Trin., but its pharmacological effect and active mechanism are rarely reported. This study aimed to investigate the anti-inflammatory effect and action mechanism of Aureusidin in LPS-induced mouse macrophage RAW264.7 cells. The results suggested that lipopolysaccharide (LPS)-induced nitric oxide (NO), tumor necrosis factor-α (TNF-α) and prostaglandin E2 (PGE2) production were obviously inhibited by Aureusidin. Moreover, Aureusidin also significantly decreased the mRNA expression of various inflammatory factors in LPS-stimulated RAW264.7 cells. Furthermore, mechanistic studies showed that Aureusidin significantly inhibited nuclear transfer of nuclear factor-κB (NF-κB), while increasing the nuclear translocation of nuclear factor E2-related factor 2 (Nrf2) as well as expression of Nrf2 target genes such as heme oxygenase (HO-1) and NAD(P)H:quinone oxidoreductase 1 